<a href="https://colab.research.google.com/github/na-geciauskas/project-melichallenge/blob/main/meli_challenge_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Conectar ao banco
conn = sqlite3.connect("mercado_livre.db")

# Widgets interativos
categoria_dropdown = widgets.Dropdown(
    options=pd.read_sql("SELECT nome FROM categorias", conn)['nome'].tolist(),
    description='Categoria:'
)

preco_slider = widgets.IntRangeSlider(
    value=[0, 2000],
    min=0,
    max=5000,
    step=100,
    description='Faixa de preço:'
)

# Função de atualização
def update_dashboard(categoria, preco_range):
    query = f"""
    SELECT p.titulo, p.preco, p.vendidos, p.condicao, c.nome as categoria
    FROM produtos p
    JOIN produto_categoria pc ON p.id = pc.produto_id
    JOIN categorias c ON pc.categoria_id = c.categoria_id
    WHERE c.nome = '{categoria}'
    AND p.preco BETWEEN {preco_range[0]} AND {preco_range[1]}
    """

    df = pd.read_sql(query, conn)

    # Gráfico 1: Distribuição de preços
    fig1 = px.histogram(df, x='preco', nbins=20,
                       title=f'Distribuição de Preços - {categoria}')

    # Gráfico 2: Preço vs Vendidos
    fig2 = px.scatter(df, x='preco', y='vendidos', color='condicao',
                     title='Relação Preço x Unidades Vendidas')

    # Métricas resumidas
    avg_price = df['preco'].mean()
    total_sold = df['vendidos'].sum()

    print(f"📊 Média de preço: R${avg_price:.2f}")
    print(f"🛒 Total vendido: {total_sold} unidades")

    fig1.show()
    fig2.show()

# Dashboard interativo
widgets.interactive(
    update_dashboard,
    categoria=categoria_dropdown,
    preco_range=preco_slider
)